# How to

Check out this guide

https://github.com/rd11490/NBA_Tutorials/blob/master/shot_bins/binning_shots.py
    
change the season in url, takes a while to load and might lag, save .json file

https://stats.nba.com/stats/shotchartdetail?PlayerID=0&TeamID=0&GameID=&LeagueID=00&Season=2005-06&SeasonType=Regular+Season&Outcome=&Location=&Month=0&SeasonSegment=&DateFrom=&DateTo=&OpponentTeamID=0&VsConference=&VsDivision=&PlayerPosition=&GameSegment=&Period=0&LastNGames=0&AheadBehind=&ContextMeasure=FGM&ClutchTime=&RookieYear=

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt  

cwd = 'G:\\NBA'
#cwd = os.getcwd()

In [2]:
def loadPcl(cwd, sub, fname, mode='rb'):
    if sub == "":
        with open(os.path.join(cwd,fname ), mode) as f: 
            df = pickle.load(f)
    else:
        with open(os.path.join(cwd,sub,fname ), mode) as f: 
            df = pickle.load(f)
    return df
def writeDF(df,cwd, sub, fname):
    if sub == "":
        dir = os.path.join( cwd,fname)

    else:
        dir = os.path.join( cwd,sub,fname)

    df.to_pickle(dir)
    print("wrote " + fname)

In [69]:
reg0506 = loadPcl(cwd,"shotData","shotsReg0506.pkl")
reg1819 = loadPcl(cwd,"shotData","shotsReg1819.pkl")

#pAvgs = loadPcl(cwd,"shotData","shotsBoth.pkl")

#shots= loadPcl(cwd,"","mrg.pkl")
#len(shots)
df =reg1819

In [4]:
def agg_shots(group):
    total_shots = group['FGA'].sum() # Calculate the number of attempted shots
    made_shots = group["FGM"].sum() # Calculate the number of made shots
    percentage = round(made_shots/total_shots, 3) # Calculate the field goal percentage (round to 3 decimal places)
    
    shotpts = group["SHOT_PTS"].sum() # 0 for miss, 2 for 2pt, 3 for 3pt
    pps = round(shotpts/total_shots, 3)

    return pd.Series([made_shots, total_shots, percentage, pps]) 

In [5]:
def makeTable(df,grpByCol):
    agg = df.groupby(grpByCol).apply(agg_shots).reset_index()
    
#     if isinstance(grpByCol, list):
#         agg.columns = grpByCol + [ "FGM", "FGA", "FG%","PPS"]
#     else:
#         agg.columns = [grpByCol, "FGM", "FGA", "FG%","PPS"]
    agg.columns = (grpByCol + [ "FGM", "FGA", "FG%","PPS"])
    agg = agg.sort_values("FGA",ascending=False)
    
    return agg

In [6]:
def makeTables(df):
    n_shots = df.FGA.sum()
    
    aggType = makeTable(df,["SHOT_TYPE"])
    aggZone = makeTable(df,["SHOT_ZONE"])
    aggSvSt = makeTable(df,["SHOT_VALUE","SHOT_TYPE"])
    aggZT = makeTable(df,["SHOT_ZONE","SHOT_TYPE"])
    
    aggPVT = makeTable(df,["PLAYER_NAME","SHOT_VALUE","SHOT_TYPE"])
    aggPZ = makeTable(df,["PLAYER_NAME","SHOT_ZONE"])
    aggPZT = makeTable(df,["PLAYER_NAME","SHOT_ZONE","SHOT_TYPE"])
    aggPV = makeTable(df,["PLAYER_NAME","SHOT_VALUE"])
    aggPVZT = makeTable(df,["PLAYER_NAME","SHOT_VALUE","SHOT_ZONE","SHOT_TYPE"])
    
    df2= df[df.SHOT_TYPE.isin(['Turnaround Jump Shot',"Fadeaway Jump Shot","Turnaround Fadeaway shot","Turnaround Fadeaway Bank Jump Shot"])]
    aggTF = makeTable(df2,["PLAYER_NAME","SHOT_ZONE"])


    return aggType,aggZone, aggSvSt,aggZT,aggPVT, aggPZ, aggPZT, aggPV,aggPVZT,aggTF

In [7]:
cols = ["Jump Shot","Pullup Jump shot","Step Back Jump shot",
        'Turnaround Jump Shot',"Fadeaway Jump Shot","Turnaround Fadeaway shot","Turnaround Fadeaway Bank Jump Shot"]#'Running Jump Shot', 'Running Pull-Up Jump Shot',

# Turnarounds and fadeaways  
Comparing 05/06 vs 18/19

In [8]:
# league 1819
aggType,aggZone, aggSvSt,aggZT,aggPVT, aggPZ, aggPZT, aggPV,aggPVZT,aggTF = makeTables(df)

In [ ]:
#aggZT[(aggZT.SHOT_TYPE.isin(cols))&(aggZT.SHOT_ZONE=="MR")]

In [ ]:
aggZT[(aggZT.SHOT_ZONE=="MR")].head(10)

In [ ]:
aggZT[(aggZT.SHOT_ZONE=="MR")].FGA.sum()

In [ ]:
aggZone

In [ ]:
aggPVT[(aggPVT.SHOT_VALUE==2) & ((aggPVT.SHOT_TYPE=='Turnaround Jump Shot') | (aggPVT.SHOT_TYPE=='Fadeaway Jump Shot')) & (aggPVT.FGA>30)].head(10)

In [ ]:
#aggPVT[~(aggPVT.SHOT_TYPE.isin(["Step Back Jump shot","Pullup Jump shot"]))][["PLAYER_NAME","SHOT_VALUE","FG%"]]

In [70]:
mrg = pd.merge(aggPVT, aggPV[["PLAYER_NAME","SHOT_VALUE","FGA","FG%"]], left_on = ["PLAYER_NAME","SHOT_VALUE"], right_on=["PLAYER_NAME","SHOT_VALUE"], how ="left", suffixes=('', '_3P'))
mrg["FG%_DIFF"] = mrg["FG%"] - mrg["FG%_3P"]
mrg.head()

,PLAYER_NAME,SHOT_VALUE,SHOT_TYPE,FGM,FGA,FG%,PPS,FGA_3P,FG%_3P,FG%_DIFF
0,Stephen Curry,3,Jump Shot,196.0,549.0,0.357,1.071,810.0,0.437,-0.080
1,James Harden,3,Step Back Jump shot,210.0,540.0,0.389,1.167,1028.0,0.368,0.021
2,Eric Gordon,3,Jump Shot,194.0,527.0,0.368,1.104,600.0,0.360,0.008
3,Buddy Hield,3,Jump Shot,221.0,520.0,0.425,1.275,651.0,0.427,-0.002
4,Brook Lopez,3,Jump Shot,169.0,484.0,0.349,1.048,512.0,0.365,-0.016


In [73]:
p3 = mrg[(mrg.SHOT_VALUE==3) & ((mrg.SHOT_TYPE=="Step Back Jump shot") | (mrg.SHOT_TYPE=="Pullup Jump shot"))& (mrg.FGA>70) ].sort_values("FG%_DIFF",ascending=False) #& (aggPS.FGA>30)& (aggPS["FG%"]>0.42)
p3[["PLAYER_NAME","SHOT_VALUE","SHOT_TYPE","FGM","FGA","FG%","FG%_3P","FG%_DIFF","FGA_3P"]]

,PLAYER_NAME,SHOT_VALUE,SHOT_TYPE,FGM,FGA,FG%,FG%_3P,FG%_DIFF,FGA_3P
609,Klay Thompson,3,Pullup Jump shot,44.0,72.0,0.611,0.402,0.209,599.0
274,Stephen Curry,3,Pullup Jump shot,88.0,140.0,0.629,0.437,0.192,810.0
422,Blake Griffin,3,Pullup Jump shot,42.0,101.0,0.416,0.362,0.054,522.0
161,D'Angelo Russell,3,Pullup Jump shot,80.0,194.0,0.412,0.369,0.043,635.0
397,Spencer Dinwiddie,3,Pullup Jump shot,40.0,107.0,0.374,0.335,0.039,370.0
534,Kevin Durant,3,Pullup Jump shot,32.0,82.0,0.390,0.353,0.037,388.0
212,Luka Doncic,3,Step Back Jump shot,60.0,166.0,0.361,0.327,0.034,514.0
110,Kemba Walker,3,Pullup Jump shot,92.0,238.0,0.387,0.356,0.031,731.0
62,Damian Lillard,3,Pullup Jump shot,114.0,287.0,0.397,0.369,0.028,643.0
495,Tim Hardaway Jr.,3,Pullup Jump shot,32.0,88.0,0.364,0.340,0.024,477.0


In [65]:
tf = aggTF[(aggTF.SHOT_ZONE =="MR") ]#.head(10)

In [66]:
mr = aggPZ[(aggPZ.SHOT_ZONE =="MR")  ]#.head(10)

In [67]:
js = aggPZT[(aggPZT.SHOT_ZONE =="MR") & (aggPZT.SHOT_TYPE == "Jump Shot")  ]#.head(10)

In [68]:
mrg = pd.merge(tf, js, left_on = ["PLAYER_NAME"], right_on=["PLAYER_NAME"], how ="left", suffixes=('', '_JS'))
mrg["FG%_DIFF"] = mrg["FG%"] - mrg["FG%_JS"]
#mrg.sort_values("FG%_DIFF",ascending=False)#.head(15)
mrg[["PLAYER_NAME","FGM","FGA","FG%","FG%_JS","FG%_DIFF","FGM_JS","FGA_JS"]].head(15)#.sort_values("FG%_DIFF",ascending=False)#.head(15)
#mrg.head()

,PLAYER_NAME,FGM,FGA,FG%,FG%_JS,FG%_DIFF,FGM_JS,FGA_JS
0,LaMarcus Aldridge,99.0,216.0,0.458,0.473,-0.015,114.0,241.0
1,Kevin Durant,89.0,153.0,0.582,0.432,0.150,54.0,125.0
2,DeMar DeRozan,45.0,121.0,0.372,0.451,-0.079,51.0,113.0
3,Dwyane Wade,37.0,112.0,0.330,0.167,0.163,5.0,30.0
4,Klay Thompson,49.0,96.0,0.510,0.325,0.185,49.0,151.0
5,Lou Williams,36.0,88.0,0.409,0.280,0.129,14.0,50.0
6,Kyrie Irving,47.0,86.0,0.547,0.480,0.067,24.0,50.0
7,LeBron James,30.0,78.0,0.385,0.541,-0.156,20.0,37.0
8,Kawhi Leonard,31.0,70.0,0.443,0.475,-0.032,28.0,59.0
9,Jayson Tatum,24.0,68.0,0.353,0.188,0.165,6.0,32.0


# 05/06


In [ ]:
len(reg0506)

In [9]:
aggType06,aggZone06, aggSvSt06,aggZT06,aggPVT06, aggPZ06, aggPZT06, aggPV06, aggPVZT06,aggTF06 = makeTables(reg0506)

In [ ]:
#aggZT06[(aggZT06.SHOT_TYPE.isin(cols))&(aggZT06.SHOT_ZONE=="MR")]
#aggZT06[(aggZT06.SHOT_TYPE.isin(cols))&(aggZT06.SHOT_ZONE=="MR")]

In [ ]:
aggZT06[(aggZT06.SHOT_ZONE=="MR")]

In [ ]:
aggZT06[(aggZT06.SHOT_ZONE=="MR")].FGA.sum()

In [ ]:
# 3P Shot types

In [ ]:
js = aggPZT[(aggPZT.SHOT_ZONE =="MR") & (aggPZT.SHOT_TYPE.isin(['Jump Shot'])) ]#.head(10)

tf = aggPZT[(aggPZT.SHOT_ZONE =="MR") & (aggPZT.SHOT_TYPE.isin(['Turnaround Jump Shot','Fadeaway Jump Shot'])& (aggPZT.FGA>30)) ]#.head(10)

mrg = pd.merge(tf, js, left_on = ["PLAYER_NAME","SHOT_ZONE"], right_on=["PLAYER_NAME","SHOT_ZONE"], how ="left", suffixes=('', '_JS'))
mrg["FG%_DIFF"] = mrg["FG%"] - mrg["FG%_JS"]
#mrg.sort_values("FG%_DIFF",ascending=False)#.head(15)
mrg.head(15)#.sort_values("FG%_DIFF",ascending=False)#.head(15)

In [ ]:
aggZone06

In [ ]:
aggPVT06[(aggPVT06.SHOT_VALUE==2) & (aggPVT06.SHOT_TYPE.isin(['Jump Shot','Turnaround Jump Shot','Fadeaway Jump Shot'])) & (aggPVT06.FGA>30)].head(10)

In [ ]:
js = aggPZT06[(aggPZT06.SHOT_ZONE =="MR") & (aggPZT06.SHOT_TYPE.isin(['Jump Shot'])) ]#.head(10)

In [ ]:
tf = aggPZT06[(aggPZT06.SHOT_ZONE =="MR") & (aggPZT06.SHOT_TYPE.isin(['Turnaround Jump Shot','Fadeaway Jump Shot'])& (aggPZT06.FGA>30)) ]#.head(10)

In [ ]:
mrg = pd.merge(tf, js, left_on = ["PLAYER_NAME","SHOT_ZONE"], right_on=["PLAYER_NAME","SHOT_ZONE"], how ="left", suffixes=('', '_JS'))
mrg["FG%_DIFF"] = mrg["FG%"] - mrg["FG%_JS"]
#mrg.sort_values("FG%_DIFF",ascending=False)#.head(15)
mrg[["PLAYER_NAME","SHOT_TYPE","FGM","FGA","FG%","FG%_JS","FG%_DIFF","FGM_JS","FGA_JS","SHOT_TYPE_JS"]].head(15)#.sort_values("FG%_DIFF",ascending=False)#.head(15)

In [51]:
tf = aggTF06[(aggTF06.SHOT_ZONE =="MR") ]#.head(10)

In [57]:
mr = aggPZ06[(aggPZ06.SHOT_ZONE =="MR")  ]#.head(10)

In [58]:
js = aggPZT06[(aggPZT06.SHOT_ZONE =="MR") & (aggPZT06.SHOT_TYPE == "Jump Shot")  ]#.head(10)

In [64]:
mrg = pd.merge(tf, js, left_on = ["PLAYER_NAME"], right_on=["PLAYER_NAME"], how ="left", suffixes=('', '_JS'))
mrg["FG%_DIFF"] = mrg["FG%"] - mrg["FG%_JS"]
#mrg.sort_values("FG%_DIFF",ascending=False)#.head(15)
mrg[["PLAYER_NAME","FGM","FGA","FG%","FG%_JS","FG%_DIFF","FGM_JS","FGA_JS"]].head(15)#.sort_values("FG%_DIFF",ascending=False)#.head(15)
#mrg.head()

,PLAYER_NAME,FGM,FGA,FG%,FG%_JS,FG%_DIFF,FGM_JS,FGA_JS
0,Dirk Nowitzki,101.0,164.0,0.616,0.439,0.177,260.0,592.0
1,Kobe Bryant,106.0,153.0,0.693,0.355,0.338,281.0,792.0
2,Kevin Garnett,72.0,131.0,0.550,0.451,0.099,215.0,477.0
3,Jason Richardson,58.0,112.0,0.518,0.296,0.222,80.0,270.0
4,Yao Ming,54.0,100.0,0.540,0.252,0.288,41.0,163.0
5,LeBron James,44.0,93.0,0.473,0.348,0.125,178.0,511.0
6,Carmelo Anthony,38.0,73.0,0.521,0.398,0.123,225.0,566.0
7,Elton Brand,53.0,73.0,0.726,0.417,0.309,221.0,530.0
8,Dwyane Wade,29.0,70.0,0.414,0.359,0.055,163.0,454.0
9,Chris Bosh,24.0,63.0,0.381,0.455,-0.074,212.0,466.0


In [62]:
mr.head()

,PLAYER_NAME,SHOT_ZONE,FGM,FGA,FG%,PPS
1373,Kobe Bryant,MR,414.0,976.0,0.424,0.848
705,Dirk Nowitzki,MR,367.0,762.0,0.482,0.963
44,Allen Iverson,MR,308.0,754.0,0.408,0.817
1960,Richard Hamilton,MR,308.0,704.0,0.438,0.875
467,Chris Webber,MR,256.0,666.0,0.384,0.769


In [61]:
js.head()

,PLAYER_NAME,SHOT_ZONE,SHOT_TYPE,FGM,FGA,FG%,PPS
5510,Kobe Bryant,MR,Jump Shot,281.0,792.0,0.355,0.710
188,Allen Iverson,MR,Jump Shot,235.0,669.0,0.351,0.703
8052,Richard Hamilton,MR,Jump Shot,262.0,635.0,0.413,0.825
6880,Mike Bibby,MR,Jump Shot,265.0,628.0,0.422,0.844
1870,Chris Webber,MR,Jump Shot,219.0,602.0,0.364,0.728
